In [230]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
from dotenv import load_dotenv
import os


load_dotenv()


hostname=os.getenv("hostname")
database=os.getenv("database")
username=os.getenv("username")
pwd=os.getenv("pwd")
port_id=os.getenv("port_id")
host = os.getenv("host")



In [231]:
conn_string = f"postgresql://{username}:{pwd}@{host}:{port_id}/{database}"
db =  create_engine(url=conn_string)

In [232]:
transactions = pd.read_sql("SELECT * FROM transactions", db)
historicalPricesAG = pd.read_sql('SELECT * FROM "historical_pricesA_G" ', db)
historicalPricesHR = pd.read_sql('SELECT * FROM "historical_pricesH_R" ', db)
historicalPricesSZ = pd.read_sql('SELECT * FROM "historical_pricesS_Z" ', db)
senators = pd.read_sql("SELECT * FROM senator_info", db)

senators.head()

,row,senator,senatorId,party
0,1,Thomas H Tuberville,100,Republican
1,2,Thomas R Carper,101,Democratic
2,3,Daniel S Sullivan,102,Republican
3,4,Rick Scott,103,Republican
4,5,John Boozman,104,Republican


In [233]:
def lookupIdByName(nameString):
    nameMatch = senators[senators['senator'].str.contains(nameString)]

    if len(nameMatch)>1:
        return nameMatch
    else: 
        return nameMatch['senatorId']


lookupIdByName("Tuber")

0    100
Name: senatorId, dtype: int64

In [234]:
tub  = transactions[transactions['senatorId']==100]
tub.ticker.value_counts().head(15)


--      88
ECOM    30
CLF     28
X       26
PYPL    24
OXY     20
INTC    19
MSFT    15
SSYS    15
BABA    14
AA      14
CALX    10
QCOM     9
F        8
NU       6
Name: ticker, dtype: int64

In [235]:
tub[(tub['ticker']=='QCOM') & (tub['type']=='Purchase') 
& (tub['asset_type']=='Stock')
]


,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId
261,06/16/2022,Joint,QCOM,QUALCOMM Incorporated - Common Stock,Stock,Purchase,"$250,001 - $500,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/3...,07/13/2022,261,100
1007,03/29/2021,Joint,QCOM,QUALCOMM Incorporated,Stock,Purchase,"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/3...,07/23/2021,1002,100


In [236]:

def reformatDate(transDate):
    listDate  = transDate.split("/")
    return f"{listDate[2]}-{listDate[0]}-{listDate[1]}"

reformatDate('03/29/2021')


'2021-03-29'

In [237]:
for frame in [historicalPricesAG, historicalPricesHR, historicalPricesSZ]:
    frame['date'] = pd.to_datetime(frame['date'])

qc = historicalPricesHR[['QCOM', 'SPY', 'date']]
temp = qc[qc['date']>= '2021-03-29']
temp


,QCOM,SPY,date
0,NaN,NaN,2022-11-15
1,121.43,398.51,2022-11-11
2,106.69,376.35,2022-11-04
3,119.21,389.02,2022-10-28
4,115.74,374.29,2022-10-21
...,...,...,...
81,138.80,417.30,2021-04-30
82,135.43,416.74,2021-04-23
83,138.21,417.26,2021-04-16
84,140.57,411.49,2021-04-09


In [238]:
temp = temp.sort_values(by="date")
temp.head(10)

,QCOM,SPY,date
85,137.79,400.61,2021-04-01
84,140.57,411.49,2021-04-09
83,138.21,417.26,2021-04-16
82,135.43,416.74,2021-04-23
81,138.80,417.30,2021-04-30
80,137.85,422.12,2021-05-07
79,130.15,416.58,2021-05-14
78,131.46,414.94,2021-05-21
77,134.54,420.04,2021-05-28
76,134.34,422.60,2021-06-04


In [239]:


def createGrowthColumn(ticker, frame):

    pricesList = np.array(frame[ticker])
    pctChanges = np.diff(pricesList)/pricesList[:-1]

    currentInvestment , initialInvestment= 100, 100
    investmentValues = []

    for c in pctChanges[1:]: 
        currentInvestment = currentInvestment*(1+c) 
        investmentValues.append(currentInvestment)

    pctChanges = [np.nan ] + list(pctChanges)
    investmentValues = [ np.nan, initialInvestment ] + investmentValues

    invName= ticker+'_growth'
    pctName = ticker+'_pchange'
    frame[invName] = investmentValues
    frame[pctName] = pctChanges



createGrowthColumn('QCOM', temp)
createGrowthColumn('SPY', temp)
temp.head()


,QCOM,SPY,date,QCOM_growth,QCOM_pchange,SPY_growth,SPY_pchange
85,137.79,400.61,2021-04-01,NaN,NaN,NaN,NaN
84,140.57,411.49,2021-04-09,100.000000,0.020176,100.000000,0.027159
83,138.21,417.26,2021-04-16,98.321121,-0.016789,101.402221,0.014022
82,135.43,416.74,2021-04-23,96.343459,-0.020114,101.275851,-0.001246
81,138.80,417.30,2021-04-30,98.740841,0.024884,101.411942,0.001344


In [240]:
import plotly.express as px 

fig = px.line(temp, x='date', y=['QCOM_growth', 'SPY_growth' ],
 title="Growth in Qualcomm v S&P"
 )


fig.show()

What a great example! looks like there was a sharp rise in price starting in November, and by mid January Qualcomm had grown much more than the S&P. My hypothesis is that the majority of stock purchases will exhibit a similar pattern

In [241]:
import string 
print(string.ascii_uppercase)

ABCDEFGHIJKLMNOPQRSTUVWXYZ


In [256]:
def findSourceFrame(ticker):
    leadingLetter =ticker[0]

    relevantCols = ['date', ticker, 'SPY']

    if leadingLetter in 'ABCDEFG':
        rez = historicalPricesAG[relevantCols]
    elif leadingLetter in 'HIJKLMNOPQR':
        rez = historicalPricesHR[relevantCols]
    elif leadingLetter in 'STUVWXYZ':
        rez = historicalPricesSZ[relevantCols]
    else:
        return 'ticker leading letter is non-alpha'
    return rez




def generateHistoricalPricesTable(ticker, date):
    
    rez = findSourceFrame(ticker)

    rez = rez.sort_values(by="date")

    rez = rez[rez['date']>=date]

    return rez


def graphGrowthRates(frame, tickerFirst, tickerSecond='SPY'):
    yColumnList = [tickerFirst+"_growth", tickerSecond+"_growth"]
    fig = px.line(frame, x='date', y=yColumnList,
    title=f"Growth in {tickerFirst} v {tickerSecond}"
    )
    fig.show()


# putting it all together 
def graphAlpha(ticker, date):

    histPrices =  generateHistoricalPricesTable(ticker, date)
    createGrowthColumn(ticker, histPrices)
    createGrowthColumn('SPY', histPrices)

    graphGrowthRates(histPrices, ticker)
 
    


In [257]:
graphAlpha("IBM", "2015-01-05")

Our all-in-one function for graphing alpha appears to be "functioning" properly. There is no obvious alpha here, but this is just a random stock and date. Instead of picking randomly, let's graph find some actual stock purachases and thier associated dates

In [244]:
# tub[tub['']]
# tub.head()
isTuberville = transactions['senatorId']==100
isPurchase = transactions['type']=='Purchase'
isStock = transactions['asset_type']=='Stock'

tbPurchases = transactions[isTuberville & isPurchase & isStock]
tbPurchases.tail(10).loc[:, :'type']


,transaction_date,owner,ticker,asset_description,asset_type,type
1034,03/29/2021,Joint,ATHM,Autohome Inc.,Stock,Purchase
1035,03/29/2021,Joint,AME,"AMETEK, Inc.",Stock,Purchase
1036,03/29/2021,Joint,GOOGL,Alphabet Inc.,Stock,Purchase
1039,03/16/2021,Joint,AA,Alcoa Corporation,Stock,Purchase
1040,03/16/2021,Joint,AA,Alcoa Corporation,Stock,Purchase
1043,03/12/2021,Joint,CLF,Cleveland-Cliffs Inc.,Stock,Purchase
1056,04/19/2021,Joint,CLF,Cleveland-Cliffs Inc.,Stock,Purchase
1059,04/16/2021,Joint,ECOM,ChannelAdvisor Corporation,Stock,Purchase
1069,04/07/2021,Joint,OXY,Occidental Petroleum Corporation,Stock,Purchase
1082,05/06/2021,Joint,X,United States Steel Corporation,Stock,Purchase


In [245]:
graphAlpha("OXY", '2021-06-25')

In [246]:
alDate = reformatDate("03/16/2021")
graphAlpha('AA', alDate)


In [247]:

graphAlpha("GOOGL", reformatDate("03/29/2021"))

After the initial panic had subsided from assuming my data is incorrect, it appears GOOGL had a stock split in July. Lets correct this and post to the db. 

In [248]:
gf = generateHistoricalPricesTable("GOOGL", reformatDate("03/29/2021"))
gf[gf['date']>='2022-07-01'].head()



,date,GOOGL,SPY
20,2022-07-01,2174.75,381.24
19,2022-07-08,2387.07,388.67
18,2022-07-15,2235.55,385.13
17,2022-07-22,107.90,395.09
16,2022-07-29,116.32,411.99


In [249]:
historicalPricesAG[['GOOGL', 'date']].tail()

,GOOGL,date
540,576.52,2012-07-13
541,585.98,2012-07-06
542,580.07,2012-06-29
543,571.48,2012-06-22
544,564.51,2012-06-15


In [253]:
# 03/16/2021